In [1]:
from __future__ import division, print_function
%matplotlib inline

In [2]:
import sympy
from sympy import Matrix, eye, symbols, sin, cos, zeros
from sympy.physics.mechanics import *
from IPython.display import display
sympy.init_printing(use_latex='mathjax')


# Quaternion Math Functions

In [3]:
def expq(n):
    n *= 0.5
    nNorm = n.norm()
    qn = Matrix([cos(nNorm),n/nNorm*sin(nNorm)])
    return qn

def quat2dcm(q):
    """
    Convert quaternion to DCM
    """
    
    # Extract components
    w = q[0]
    x = q[1]
    y = q[2]
    z = q[3]
    
    # Reduce repeated calculations
    ww = w*w
    xx = x*x
    yy = y*y
    zz = z*z  
    wx = w*x
    wy = w*y
    wz = w*z
    xy = x*y
    xz = x*z
    yz = y*z
    
    # Build Direction Cosine Matrix (DCM)   
    dcm = Matrix([
        [ww + xx - yy - zz,       2*(xy - wz),       2*(xz + wy)],
        [      2*(xy + wz), ww - xx + yy - zz,       2*(yz - wx)],
        [      2*(xz - wy),       2*(yz + wx), ww - xx - yy + zz]
    ])
    return dcm

def skew3(v):
    vx,vy,vz = v
    out = Matrix([[  0, -vz,   vy],
                  [ vz,   0,  -vx],
                  [-vy,  vx,    0]])
    return out

def skew4Left(v):
    if len(v)==3:
        v = Matrix.vstack(zeros(1),v)
    w,x,y,z = v
    out = Matrix([
            [w, -x, -y, -z],
            [x,  w, -z,  y],
            [y,  z,  w, -x],
            [z, -y,  x,  w],
        ])        
    return out

def skew4Right(v):
    if len(v)==3:
        v = Matrix.vstack(zeros(1),v)
    w,x,y,z = v
    out = Matrix([
            [w, -x, -y, -z],
            [x,  w,  z, -y],
            [y, -z,  w,  x],
            [z,  y, -x,  w],
        ])      
    return out


def quatConj(q):
    q_out = Matrix(q[:])
    q_out = q_out.T*sympy.diag(1,-1,-1,-1)
    q_out = q_out.T

    return q_out

def qRot(q,v):
    qPrime = quatConj(q)
    v = Matrix.vstack(zeros(1),v)
    vout = skew4Left(q)*skew4Right(qPrime)*v
    return Matrix(vout[1:])

# Inertia Tensor

In [4]:
def build_inertia_tensor(Ivec):
    Ixx,Iyy,Izz,Ixy,Ixz,Iyz = Ivec
    
    Imat = zeros(3,3)
    Imat[0,0] = Ixx
    Imat[0,1] = Ixy
    Imat[0,2] = Ixz
    Imat[1,0] = Ixy
    Imat[1,1] = Iyy
    Imat[1,2] = Iyz
    Imat[2,0] = Ixz
    Imat[2,1] = Iyz
    Imat[2,2] = Izz
    
    return Imat

# 6DOF EOM using general body frame Force and Moment

Define Sympy Symbols

In [5]:
rx,ry,rz = symbols('r_x r_y r_z')
vx,vy,vz = symbols('v_x v_y v_z')
qw, qx, qy, qz = symbols('q_w, q_x, q_y, q_z')
wx, wy, wz = symbols('w_x, w_y, w_z')
Ixx, Iyy, Izz, Ixy, Ixz, Iyz = symbols('I_xx, I_yy, I_zz, I_xy, I_xz, I_yz')
Mx, My, Mz = symbols('M_x, M_y, M_z') 
Fbx, Fby, Fbz = symbols('F_x, F_y, F_z') 
m,g = symbols('m g')
L = symbols('L') # Quadcopter arm length

Setup Vectors

In [6]:
r_BwrtLexpL = Matrix([rx,ry,rz])
v_BwrtLexpL = Matrix([vx,vy,vz])
q_toLfromB  = Matrix([qw,qx,qy,qz])
wb          = Matrix([wx,wy,wz])
Fb          = Matrix([Fbx,Fby,Fbz])
Mb          = Matrix([Mx,My,Mz])

## Build Inertia Tensor

In [7]:
Ixy = 0
Ixz = 0
Iyz = 0
Ivec = Ixx,Iyy,Izz,Ixy,Ixz,Iyz
inertiaTensor = build_inertia_tensor(Ivec)
display(inertiaTensor)
display(inertiaTensor.inv())

⎡Iₓₓ   0     0  ⎤
⎢               ⎥
⎢ 0   I_yy   0  ⎥
⎢               ⎥
⎣ 0    0    I_zz⎦

⎡ 1             ⎤
⎢───   0     0  ⎥
⎢Iₓₓ            ⎥
⎢               ⎥
⎢      1        ⎥
⎢ 0   ────   0  ⎥
⎢     I_yy      ⎥
⎢               ⎥
⎢            1  ⎥
⎢ 0    0    ────⎥
⎣           I_zz⎦

Gravity Vector in local frame (NED)

In [8]:
g_expL = Matrix([0,0,g])

## Body Forces & Moments

In [9]:
# Motor speeds
wm1, wm2, wm3, wm4 = symbols('w_m1, w_m2, w_m3, w_m4')
# Motor force and moment coefficients
kF, kM = symbols('k_F, k_M')

# Motor Thrust and Torque
Fm1 = kF*wm1**2
Fm2 = kF*wm2**2
Fm3 = kF*wm3**2
Fm4 = kF*wm4**2
Mm1 = kM*wm1**2
Mm2 = kM*wm2**2
Mm3 = kM*wm3**2
Mm4 = kM*wm4**2

# Calc Body Forces due to motors
Fb[0] = 0
Fb[1] = 0
Fb[2] = -(Fm1+Fm2+Fm3+Fm4)

# Calc Body Moments dut to motors
Mb[0] = L*(Fm4-Fm2)
Mb[1] = L*(Fm1-Fm3)
Mb[2] = Mm2 + Mm4 - Mm1 -Mm3

print('Fb')
display(Fb)
print('Mb')
display(Mb)


Fb


⎡                     0                     ⎤
⎢                                           ⎥
⎢                     0                     ⎥
⎢                                           ⎥
⎢         2          2          2          2⎥
⎣- k_F⋅wₘ₁  - k_F⋅wₘ₂  - k_F⋅wₘ₃  - k_F⋅wₘ₄ ⎦

Mb


⎡           ⎛         2          2⎞         ⎤
⎢         L⋅⎝- k_F⋅wₘ₂  + k_F⋅wₘ₄ ⎠         ⎥
⎢                                           ⎥
⎢            ⎛       2          2⎞          ⎥
⎢          L⋅⎝k_F⋅wₘ₁  - k_F⋅wₘ₃ ⎠          ⎥
⎢                                           ⎥
⎢         2          2          2          2⎥
⎣- k_M⋅wₘ₁  + k_M⋅wₘ₂  - k_M⋅wₘ₃  + k_M⋅wₘ₄ ⎦

## Sum of forces

In [10]:
a_BwrtLexpL = 1/m*qRot(q_toLfromB,Fb) + g_expL

print('a_BwrtLexpL')
display(a_BwrtLexpL)

a_BwrtLexpL


⎡                           ⎛         2          2          2          2⎞    ⎤
⎢    (2⋅q_w⋅q_y + 2⋅qₓ⋅q_z)⋅⎝- k_F⋅wₘ₁  - k_F⋅wₘ₂  - k_F⋅wₘ₃  - k_F⋅wₘ₄ ⎠    ⎥
⎢    ────────────────────────────────────────────────────────────────────    ⎥
⎢                                     m                                      ⎥
⎢                                                                            ⎥
⎢                           ⎛         2          2          2          2⎞    ⎥
⎢   (-2⋅q_w⋅qₓ + 2⋅q_y⋅q_z)⋅⎝- k_F⋅wₘ₁  - k_F⋅wₘ₂  - k_F⋅wₘ₃  - k_F⋅wₘ₄ ⎠    ⎥
⎢   ─────────────────────────────────────────────────────────────────────    ⎥
⎢                                     m                                      ⎥
⎢                                                                            ⎥
⎢    ⎛   2     2      2      2⎞ ⎛         2          2          2          2⎞⎥
⎢    ⎝q_w  - qₓ  - q_y  + q_z ⎠⋅⎝- k_F⋅wₘ₁  - k_F⋅wₘ₂  - k_F⋅wₘ₃  - k_F⋅wₘ₄ ⎠⎥
⎢g + ───────────────────────────────────────────────

## Sum of moments

In [11]:
wbDot = inertiaTensor.inv() * (-skew3(wb)*inertiaTensor*wb + Mb)

print('wbDot')
display(wbDot)

wbDot


⎡                                      ⎛         2          2⎞       ⎤
⎢      I_yy⋅w_y⋅w_z - I_zz⋅w_y⋅w_z + L⋅⎝- k_F⋅wₘ₂  + k_F⋅wₘ₄ ⎠       ⎥
⎢      ───────────────────────────────────────────────────────       ⎥
⎢                                Iₓₓ                                 ⎥
⎢                                                                    ⎥
⎢                                      ⎛       2          2⎞         ⎥
⎢        -Iₓₓ⋅wₓ⋅w_z + I_zz⋅wₓ⋅w_z + L⋅⎝k_F⋅wₘ₁  - k_F⋅wₘ₃ ⎠         ⎥
⎢        ───────────────────────────────────────────────────         ⎥
⎢                                I_yy                                ⎥
⎢                                                                    ⎥
⎢                                  2          2          2          2⎥
⎢Iₓₓ⋅wₓ⋅w_y - I_yy⋅wₓ⋅w_y - k_M⋅wₘ₁  + k_M⋅wₘ₂  - k_M⋅wₘ₃  + k_M⋅wₘ₄ ⎥
⎢────────────────────────────────────────────────────────────────────⎥
⎣                                I_zz                                ⎦

## Quaternion Kinematic Equation

In [12]:
qDot = 0.5*skew4Left(q_toLfromB) * Matrix.vstack(zeros(1),wb)
# 0.5*skew4Left(q_toLfromB)[:,1:] * wb

#display(skew4Left(q_toLfromB))
#display(skew4Left(q_toLfromB)[:,1:])
#display(0.5*skew4Left(q_toLfromB)[:,1:] * wb)

print('qDot')
display(qDot)

qDot


⎡-0.5⋅qₓ⋅wₓ - 0.5⋅q_y⋅w_y - 0.5⋅q_z⋅w_z⎤
⎢                                      ⎥
⎢0.5⋅q_w⋅wₓ + 0.5⋅q_y⋅w_z - 0.5⋅q_z⋅w_y⎥
⎢                                      ⎥
⎢0.5⋅q_w⋅w_y - 0.5⋅qₓ⋅w_z + 0.5⋅q_z⋅wₓ ⎥
⎢                                      ⎥
⎣0.5⋅q_w⋅w_z + 0.5⋅qₓ⋅w_y - 0.5⋅q_y⋅wₓ ⎦

## State and dstate vectors

In [13]:
state = Matrix([r_BwrtLexpL, v_BwrtLexpL, q_toLfromB, wb])
dstate = Matrix([
    v_BwrtLexpL,
    a_BwrtLexpL,
    qDot,
    wbDot
])
display(state.T)
display(dstate)

mprint(dstate)


[rₓ  r_y  r_z  vₓ  v_y  v_z  q_w  qₓ  q_y  q_z  wₓ  w_y  w_z]

⎡                                     vₓ                                     ⎤
⎢                                                                            ⎥
⎢                                    v_y                                     ⎥
⎢                                                                            ⎥
⎢                                    v_z                                     ⎥
⎢                                                                            ⎥
⎢                           ⎛         2          2          2          2⎞    ⎥
⎢    (2⋅q_w⋅q_y + 2⋅qₓ⋅q_z)⋅⎝- k_F⋅wₘ₁  - k_F⋅wₘ₂  - k_F⋅wₘ₃  - k_F⋅wₘ₄ ⎠    ⎥
⎢    ────────────────────────────────────────────────────────────────────    ⎥
⎢                                     m                                      ⎥
⎢                                                                            ⎥
⎢                           ⎛         2          2          2          2⎞    ⎥
⎢   (-2⋅q_w⋅qₓ + 2⋅q_y⋅q_z)⋅⎝- k_F⋅wₘ₁  - k_F⋅wₘ₂  -

Matrix([
[                                                                                               v_x],
[                                                                                               v_y],
[                                                                                               v_z],
[                (2*q_w*q_y + 2*q_x*q_z)*(-k_F*w_m1**2 - k_F*w_m2**2 - k_F*w_m3**2 - k_F*w_m4**2)/m],
[               (-2*q_w*q_x + 2*q_y*q_z)*(-k_F*w_m1**2 - k_F*w_m2**2 - k_F*w_m3**2 - k_F*w_m4**2)/m],
[g + (q_w**2 - q_x**2 - q_y**2 + q_z**2)*(-k_F*w_m1**2 - k_F*w_m2**2 - k_F*w_m3**2 - k_F*w_m4**2)/m],
[                                                          -0.5*q_x*w_x - 0.5*q_y*w_y - 0.5*q_z*w_z],
[                                                           0.5*q_w*w_x + 0.5*q_y*w_z - 0.5*q_z*w_y],
[                                                           0.5*q_w*w_y - 0.5*q_x*w_z + 0.5*q_z*w_x],
[                                                           0.5*q_w*w_z +

In [14]:
from sympy.physics.mechanics import *
from sympy import sin, cos, symbols, Matrix, solve

In [15]:
# Inertial Reference Frame
N = ReferenceFrame('N')

# Define world corredinate origin
O = Point('O')
O.set_vel(N, 0.0)